#1) import libraries

In [0]:
from google.colab import drive

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from keras.preprocessing.image import ImageDataGenerator, load_img

#2) Map my google Drive to import dataset

In [0]:
drive.mount('/content/gdrive')
main_path = 'gdrive/My Drive/covid-19-dataset/'

train_path = 'gdrive/My Drive/covid-19-dataset/train'
test_path = 'gdrive/My Drive/covid-19-dataset/test'
val_path = 'gdrive/My Drive/covid-19-dataset/val'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#3) Creating the model

In [0]:
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(320,320,3)),
    layers.MaxPooling2D(pool_size= (2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size= (2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size= (2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size= (2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size= (2,2)),
    layers.Flatten(),
    layers.Dense(activation='relu', units=1280),
    layers.Dense(activation='sigmoid', units=1)
    ])

model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

#4) Generating more Data

In [0]:
batch=10
train_datagen= ImageDataGenerator(rescale= 1./255, 
                                  shear_range=0.2, 
                                  zoom_range= 0.2, 
                                  rotation_range=20, 
                                  horizontal_flip=True)

test_datagen= ImageDataGenerator(rescale= 1./255)

training_set= train_datagen.flow_from_directory(train_path, target_size= (320,320), batch_size=batch, class_mode='binary')
test_set= test_datagen.flow_from_directory(test_path, target_size= (320,320), batch_size=batch, class_mode='binary')
val_set= test_datagen.flow_from_directory(val_path, target_size= (320,320), batch_size=batch, class_mode='binary')

model.summary()

Found 465 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 318, 318, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 159, 159, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 157, 157, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 78, 78, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 76, 76, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 38, 38, 64)        0         

#5) Training the Model

In [0]:
history= model.fit_generator(training_set, steps_per_epoch= 46, epochs = 15, validation_data= val_set, validation_steps=5)

Epoch 1/15
46/46 [==============================] - 75s 2s/step - loss: 0.6565 - accuracy: 0.6176 - val_loss: 0.5269 - val_accuracy: 0.6800
Epoch 2/15
46/46 [==============================] - 75s 2s/step - loss: 0.4382 - accuracy: 0.7956 - val_loss: 0.5664 - val_accuracy: 0.7400
Epoch 3/15
46/46 [==============================] - 74s 2s/step - loss: 0.3379 - accuracy: 0.8681 - val_loss: 0.4299 - val_accuracy: 0.8200
Epoch 4/15
46/46 [==============================] - 74s 2s/step - loss: 0.3114 - accuracy: 0.8857 - val_loss: 0.5950 - val_accuracy: 0.7800
Epoch 5/15
46/46 [==============================] - 74s 2s/step - loss: 0.2928 - accuracy: 0.8725 - val_loss: 0.4936 - val_accuracy: 0.7800
Epoch 6/15
46/46 [==============================] - 74s 2s/step - loss: 0.2826 - accuracy: 0.8967 - val_loss: 0.4704 - val_accuracy: 0.8000
Epoch 7/15
46/46 [==============================] - 78s 2s/step - loss: 0.2867 - accuracy: 0.8923 - val_loss: 0.5018 - val_accuracy: 0.7600
Epoch 8/15
46/46 [==

#6) Testing the Model with the Valadation Data

In [0]:
test_accu = model.evaluate_generator(test_set, steps=20)

print('The testing accuracy is :',test_accu[1]*100, '%')

Instructions for updating:
Please use Model.evaluate, which supports generators.
The testing accuracy is : 75.0 %


#7) Export the Model with SavedModel Format to be used with tensorflow serving

In [0]:
version= 7
tf.saved_model.save(model, 'gdrive/My Drive/covid-19-dataset/saved-model/' + str(version))